<a href="https://colab.research.google.com/github/galynz/causal_inference_project/blob/master/preprocessing/train_fuonta_abuse_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO



*   Add custom metrics to the model building
*   Change the preprocess to be more accurate (or check that Tokenizer works as intended)
*   Preprocess Metadata
*   Add second head for metadata classification



In [0]:
%tensorflow_version 1.x 

# Mounting Drive For Data

(not relevant for now. Please upload data manually)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [0]:
!ls /content/drive/My\ Drive/Research

# Loading Data

In [0]:
import pandas as pd

In [0]:
features_df = pd.read_csv("./hatespeech_features.csv", lineterminator='\n', index_col=0)
labels_df = pd.read_csv("./hatespeech_labels.csv", index_col=0, dtype={"label": "category"})

# Creating Model

In [0]:
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras.backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D, GRU
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

import numpy as np

Using TensorFlow backend.


Preparing the data:

(todo: change the Tokenizer to sklearn and the required tokenizing for tweeter. In other words: more preprocess.)

In [0]:
MAX_NUM_WORDS = 20000
MAX_SEQUENCE_LENGTH = 30
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(features_df["text"])
sequences = tokenizer.texts_to_sequences(features_df["text"])
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels_df.loc[features_df.index, "label"].cat.codes))

Train-test split

In [0]:
Xtrain, Xtest, ytrain, ytest = train_test_split(data, labels)

Building the model

In [0]:
# Getting the GloVE embedding layer
import gensim.downloader as api
glove = api.load("glove-twitter-200")

[==================================================] 100.0% 758.5/758.5MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = glove.get_keras_embedding()(sequence_input)
x = GRU(128, recurrent_dropout=0.5)(embedded_sequences)
# x = Conv1D(128, 5, activation='relu')(embedded_sequences)
# x = MaxPooling1D(5)(x)
# x = Conv1D(128, 5, activation='relu')(x)
# x = GlobalMaxPooling1D()(x)
#x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_df.loc[features_df.index, "label"].cat.categories), activation='softmax')(x)











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=["accuracy"])

In [0]:
K.get_session().run(tf.local_variables_initializer())
model.fit(
    Xtrain, ytrain, batch_size=512, epochs=1000,
    validation_data=(Xtest, ytest)
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 74849 samples, validate on 24950 samples
Epoch 1/1000
74849/74849 [==============================] - 53s 708us/step - loss: 0.8576 - acc: 0.6634 - val_loss: 0.6913 - val_acc: 0.7445
Epoch 2/1000
74849/74849 [==============================] - 53s 703us/step - loss: 0.6630 - acc: 0.7543 - val_loss: 0.6259 - val_acc: 0.7660
Epoch 3/1000
74849/74849 [==============================] - 53s 708us/step - loss: 0.6056 - acc: 0.7748 - val_loss: 0.5867 - val_acc: 0.7807
Epoch 4/1000
74849/74849 [==============================] - 52s 699us/step - loss: 0.5721 - acc: 0.7880 - val_loss: 0.5665 - val_acc: 0.7890
Epoch 5/1000
74849/74849 [==============================] - 52s 695us/step - loss: 0.5517 - acc: 0.7954 - val_loss: 0.5561 - val_acc: 0.7937
Epoch 6/1000
74849/74849 [==============================] - 52s 695us/step - loss: 0.5359 - acc: 0.8017 - val_loss: 0.5511 - val_acc: 0.7956
Epoch 7/